In [2]:
import os
import cv2
import random
from pathlib import Path

# ==== CONFIG ====
dataset_dir = r"D:\Akash\Work\AI\2025\TC04\test"
images_dir = os.path.join(dataset_dir, "images")
labels_dir = os.path.join(dataset_dir, "labels")

person_class_id = 0  # Change if your "person" class has a different ID
resize_scale = 0.5   # Reduce person size to 50%
paste_nearby = True  # If True, paste resized person close to original

output_dir = dataset_dir + "_resized"
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

# ==================

def denormalize(bbox, img_w, img_h):
    """Convert normalized bbox to pixel coordinates"""
    x_center, y_center, w, h = bbox
    x1 = int((x_center - w / 2) * img_w)
    y1 = int((y_center - h / 2) * img_h)
    x2 = int((x_center + w / 2) * img_w)
    y2 = int((y_center + h / 2) * img_h)
    return x1, y1, x2, y2

def normalize(x1, y1, x2, y2, img_w, img_h):
    """Convert pixel coordinates to normalized bbox"""
    x_center = (x1 + x2) / 2 / img_w
    y_center = (y1 + y2) / 2 / img_h
    w = (x2 - x1) / img_w
    h = (y2 - y1) / img_h
    return [x_center, y_center, w, h]

for img_file in Path(images_dir).glob("*.jpg"):
    img = cv2.imread(str(img_file))
    img_h, img_w = img.shape[:2]

    label_file = Path(labels_dir) / (img_file.stem + ".txt")
    if not label_file.exists():
        continue

    with open(label_file, "r") as f:
        lines = f.readlines()

    new_lines = lines[:]
    for line in lines:
        parts = line.strip().split()
        cls_id = int(parts[0])
        if cls_id != person_class_id:
            continue

        bbox = list(map(float, parts[1:]))
        x1, y1, x2, y2 = denormalize(bbox, img_w, img_h)

        person_crop = img[y1:y2, x1:x2]
        new_w = int((x2 - x1) * resize_scale)
        new_h = int((y2 - y1) * resize_scale)

        if new_w < 1 or new_h < 1:
            continue  # skip too small

        resized_crop = cv2.resize(person_crop, (new_w, new_h), interpolation=cv2.INTER_AREA)

        # Choose a location to paste
        if paste_nearby:
            offset_x = random.randint(-30, 30)
            offset_y = random.randint(-30, 30)
            new_x1 = max(0, x1 + offset_x)
            new_y1 = max(0, y1 + offset_y)
        else:
            new_x1 = random.randint(0, img_w - new_w)
            new_y1 = random.randint(0, img_h - new_h)

        new_x2 = new_x1 + new_w
        new_y2 = new_y1 + new_h

        # Ensure it's within bounds
        if new_x2 > img_w or new_y2 > img_h:
            continue

        img[new_y1:new_y2, new_x1:new_x2] = resized_crop

        new_bbox = normalize(new_x1, new_y1, new_x2, new_y2, img_w, img_h)
        new_line = f"{person_class_id} {' '.join(map(str, new_bbox))}\n"
        new_lines.append(new_line)

    # Save image and label
    out_img_path = Path(output_dir) / "images" / img_file.name
    out_label_path = Path(output_dir) / "labels" / label_file.name

    cv2.imwrite(str(out_img_path), img)
    with open(out_label_path, "w") as f:
        f.writelines(new_lines)

print("✅ Done! Dataset with resized person instances saved at:", output_dir)


✅ Done! Dataset with resized person instances saved at: D:\Akash\Work\AI\2025\TC04\test_resized
